In [1]:
!pip install elasticsearch
!pip install python-magic

In [2]:
!pip install gdown

## Setting up the Elasticsearch

In [3]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': '7rryNDgDS1yAM3OPC7QvQA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Setting up the documents

In [4]:
# Downloading the data
"""
The data is uploaded at a google drive directory, by running this code box, 
the data will be downloaded and you will have access to it at
(enron_short/maildir)
"""
import gdown

googleDriveURL = "https://drive.google.com/file/d/10OwgK91e0lNRsrAV31sSJu7KJUujBTnE/view?usp=sharing"
output = 'enron_short.tar.gz'
gdown.download(googleDriveURL, output, quiet=False, fuzzy=True)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=10OwgK91e0lNRsrAV31sSJu7KJUujBTnE 



In [5]:
# check if the directory exists
from os import path

zipfile_path = "file_enron_short.tar.gz"
if not path.isfile(zipfile_path):
    print("the zipfile is not here, please ensure you download it first.")

In [9]:
!tar xf file_enron_short.tar.gz

In [6]:
# check if the directory exists
from os import path

documents_path = r"enron_short/maildir"
if not path.exists(documents_path):
    print("the directory is not here, please ensure you have the documnets first.")

## Your Code:

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Index all emails folder enron_short/maildir

import os
from pathlib import Path

index_name = "enron_short"

def get_content(file_path: Path):
    with file_path.open(mode='r', encoding='utf-8') as file:
        content = file.read()
        return content

def index_to_elasticsearch(content, doc_id):
    # print(index_name, doc_id, content)
    es.index(index=index_name, id=doc_id, document={'content': content})

def get_path():
    current_script_directory = os.path.dirname(os.path.abspath('elk_base_documents_template.ipynb'))
    folder_name = "enron_short"
    path = os.path.join(current_script_directory, folder_name)
    return path


def add_to_elastic():
    path = "enron_short/maildir"
    print(path)
    for subdir, dirs, files in os.walk(path):
        for file in files:
            file_path = Path(os.path.join(subdir, file))
            
            if file.endswith('.txt'):
                print(file_path)
                content = get_content(file_path)
                # Use the file name as a unique ID for the document
                doc_id = os.path.splitext(file)[0]
                #index_to_elasticsearch(content, doc_id)

add_to_elastic()

enron_short/maildir


In [ ]:
# Query for "Norwegian and University and Science and Technology” and "Norwegian University Science Technology"
index="enron_short"

resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": "Norwegian and University and Science and Technology"}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_score"])